In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!ls drive/Akshay
#images001 = glob(os.path.join("drive/app/sample/sample/images", "*.png"))

 check.py   CNN   file.txt  'Starting Kit'


In [0]:
import pandas as pd;
import numpy as np;
from sklearn.cross_validation import train_test_split;
print('loading...');
data = pd.read_csv('drive/Akshay/CNN/data.csv', sep='\t');
print('loaded');
data = np.asarray(data);
X = data[:, :-1];
y = data[:, -1];

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=42);


 

In [0]:
y_train = y_train.reshape(y_train.shape[0], 1);
y_test = y_test.reshape(y_test.shape[0], 1);
x1 = np.concatenate((X_train, y_train), axis=1);
y1 = np.concatenate((X_test, y_test), axis=1);

In [0]:
import nltk;
nltk.download('stopwords');
nltk.download('punkt');

In [0]:
import math
import pickle
from nltk.tokenize import word_tokenize;
from nltk.corpus import stopwords;
import pandas as pd;
import numpy as np;



docIDFDict = {}
avgDocLength = 0


def GetCorpus(inputfile,corpusfile):



    sw = set(stopwords.words('english'));
    f = open(inputfile,"r",encoding="utf-8")
    fw = open(corpusfile,"w",encoding="utf-8")
    
    
    dw = pd.read_csv('drive/Akshay/CNN/data.csv');
    
    dw = dw[:]
  
    i=0;
    print(i);
    strr = '';
    for line in f:
        passage = line.strip().lower().split("\t")[2]
        tok = word_tokenize(passage);
        filtered = [];
        for w in tok:
            if not w in sw:
                filtered.append(w);
        strr =' '.join(filtered);
#         fw.write(strr+'\n');
        i = i+1;
        if i%1000==0:
#             fw.write(strr+'\n');
#             strr = '';
            print(i);

    f.close()
    fw.close()



# The following IDF_Generator method reads all the passages(docs) and creates Inverse Document Frequency(IDF) scores for each unique word using below formula 
# IDF(q_i) = log((N-n(q_i)+0.5)/(n(q_i)+0.5)) where N is the total number of documents in the collection and n(q_i) is the number of documents containing q_i
# After finding IDF scores for all the words, The IDF dictionary will be saved in "docIDFDict.pickle" file in the current directory

def IDF_Generator(corpusfile, delimiter=' ', base=math.e) :

    global docIDFDict,avgDocLength

    docFrequencyDict = {}       
    numOfDocuments = 0   
    totalDocLength = 0

    for line in open(corpusfile,"r",encoding="utf-8") :
        doc = line.strip().split(delimiter)
        totalDocLength += len(doc)

        doc = list(set(doc)) # Take all unique words

        mx = 0;
        
        for word in doc : #Updates n(q_i) values for all the words(q_i)
            if word not in docFrequencyDict :
                docFrequencyDict[word] = 0
            docFrequencyDict[word] += 1
            
            mx = max(docFrequencyDict[word], mx);
        
        
          
        for word in doc:
          docFrequencyDict[word] = 0.5 + (docFrequencyDict[word]/mx)*0.5; 
          

        numOfDocuments = numOfDocuments + 1
                    

    for word in docFrequencyDict:  #Calculate IDF scores for each word(q_i)
        docIDFDict[word] = math.log((numOfDocuments + 1) / (docFrequencyDict[word]), base)
        
    avgDocLength = totalDocLength / numOfDocuments

     
#     pickle_out = open("drive/Akshay/CNN/docIDFDict_stop.pickle","wb") # Saves IDF scores in pickle file, which is optional
#     pickle.dump(docIDFDict, pickle_out)
#     pickle_out.close()





#The following GetBM25Score method will take Query and passage as input and outputs their similarity score based on the term frequency(TF) and IDF values.
# stop word removal; stemming; bm25+(b = 0.6, k=1.1, del = 0.6); double normalised tf
def GetBM25Score(Query, Passage, k1=1.1, b=0.6, delimiter=' ') :
    
    global docIDFDict,avgDocLength

    query_words= Query.strip().lower().split(delimiter)
    passage_words = Passage.strip().lower().split(delimiter)
    passageLen = len(passage_words)
    docTF = {}
    for word in set(query_words):   #Find Term Frequency of all query unique words
        docTF[word] = passage_words.count(word)
    commonWords = set(query_words) & set(passage_words)
    tmp_score = []
    for word in commonWords :   
        numer = (docTF[word] * (k1+1))   #Numerator part of BM25 Formula
        denom = ((docTF[word]) + k1*(1 - b + b*passageLen/avgDocLength)) #Denominator part of BM25 Formula 
        
#         numer1 = docTF[word];
#         denom1 = 1 - b + b*(passageLen/avgDocLength);
        
#         tm1 = 1 + math.log(math.log((numer1/denom1)+1)+1);
        
        if(word in docIDFDict) :
#             tmp_score.append(docIDFDict[word] * tm1);
              tmp_score.append(docIDFDict[word] * ((numer/denom)+0.6));

    score = sum(tmp_score)
    return score

#The following line reads each line from testfile and extracts query, passage and calculates BM25 similarity scores and writes the output in outputfile
def RunBM25OnEvaluationSet(testfile,outputfile):

    lno=0
    tempscores=[]  #This will store scores of 10 query,passage pairs as they belong to same query
    f = open(testfile,"r",encoding="utf-8")
    fw = open(outputfile,"w",encoding="utf-8")
    sw = stopwords.words('english');
    for line in f:
        tokens = line.strip().lower().split("\t")
        
        Query = tokens[1];
        tok = word_tokenize(Query);
        fil = [stemmer.stem(w) for w in tok if not w in sw]
        Query =  ' '.join(fil)
        
        
        Passage = tokens[2];
        tok = word_tokenize(Passage);
        fil = [stemmer.stem(w) for w in tok if not w in sw]
        Passage = ' '.join(fil);
        
        score = GetBM25Score(Query,Passage) 
        tempscores.append(score)
        lno+=1
        if(lno%10==0):
            tempscores = [str(s) for s in tempscores]
            scoreString = "\t".join(tempscores)
            qid = tokens[0]
            fw.write(qid+"\t"+scoreString+"\n")
            tempscores=[]
        if(lno%5000==0):
            print(lno)
    f.close()
    fw.close()


if __name__ == '__main__' :

    #inputFileName = "drive/Akshay/CNN/data.csv"   # This file should be in the following format : queryid \t query \t passage \t label \t passageid
    testFileName = "drive/Akshay/CNN/eval2_unlabelled.tsv"  # This file should be in the following format : queryid \t query \t passage \t passageid # order of the query
    corpusFileName = "drive/Akshay/CNN/stopWordRemovedStemmed.csv" 
    outputFileName = "drive/Akshay/CNN/answerPhase2_1.tsv"

    #GetCorpus(inputFileName,corpusFileName)    # Gets all the passages(docs) and stores in corpusFile. you can comment this line if corpus file is already generated
    print("Corpus File is created.")
    IDF_Generator(corpusFileName)   # Calculates IDF scores. 
    print("IDF Dictionary Generated.")
    RunBM25OnEvaluationSet(testFileName,outputFileName)
    print("Submission file created. ")



In [0]:
import pandas as pd;
dw = pd.read_csv('drive/Akshay/CNN/sampledData.csv', delimiter='\t', header=None);


In [0]:
import numpy as np;
from nltk.corpus import stopwords;
from nltk.tokenize import word_tokenize;

dw = np.asarray(dw)[:,2];

sw = stopwords.words('english');

In [0]:
dw.shape

(2096752, 5)

In [0]:
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer();

In [0]:
i=0;
for line in dw:
  tok = word_tokenize(line);
  
  fil = [stemmer.stem(w) for w in tok if not w in sw]
  fil =  ' '.join(fil)
  
  dw[i] = fil;
  i = i+1;
  if i%10000==0:
    print(i);

In [0]:
dw.shape

(2096752,)

In [0]:
np.savetxt('drive/Akshay/CNN/stopWordRemovedStemmedSampled.csv', dw, fmt='%s');

In [0]:
data = pd.read_csv('drive/Akshay/CNN/stopWordRemovedStemmedSampled.csv', delimiter='\n', header=None);

In [0]:
data.head

In [0]:
import pandas as pd;
import numpy as np;
import random;

data = pd.read_csv('drive/Akshay/CNN/data.csv', sep='\t', header=None);
print(data.shape);
print('read');
npd = np.asarray(data);

rows = npd.shape[0];

cnt=0;
i=0;
nnpd = []

(5241880, 5)
read


In [0]:
while i < rows:
  if i%1000000==0:
    print(i);
  neg = [npd[i+w] for w in range(10) if not npd[i+w][3]==1]
# 	neg = [npd[i+w] for w in range(10) if not npd[i+w][3]==1]
  rnd = random.sample(neg, 3);
# 	rnd = random.sample(neg, 3);
  j = i;
  while j<rows :
    if npd[j][3]==1:
      pos = npd[j];
      break;
    j = j+1;
  i = i+10;
  rnd.append(pos);
  nnpd.append(rnd);
nnpd = np.asarray(nnpd);

